In [ ]:
import numpy as np
import collections
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import datetime  
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)
data_path = '../data-unversioned/p1ch7/'

In [ ]:
cifar10 = datasets.CIFAR10(data_path, train=True, download= True)           # Training dataset....
cifar10_val = datasets.CIFAR10(data_path, train=False, download= True)      # Dataset for validation....

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'] # class definations...

In [ ]:
# Splitting dataset into training and validation set...

# Trainig data...
transformed_cifar10_Train = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Validation data...
transformed_cifar10_Val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
# Class "Net31" for model defination...
class Net31(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

# Traing loop function...
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader:  
            outputs = model(imgs)  
            loss = loss_fn(outputs, labels)  
            optimizer.zero_grad()         
            loss.backward()         
            optimizer.step()  
            loss_train += loss.item()  

        if epoch == 1 or epoch % 30 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader))) 

# Function calculating the validation of the model...
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad(): 
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0]  
                correct += int((predicted == labels).sum())  

        print("Accuracy {}: {:.2f}".format(name , correct / total))


In [ ]:
# Training the model "Net31" with dataset divided into minibatches...
train_mbload = torch.utils.data.DataLoader(transformed_cifar10_Train, batch_size=64, shuffle=True)  

model = Net31()  
optimizer = optim.SGD(model.parameters(), lr=1e-2)  
loss_fn = nn.CrossEntropyLoss()  

training_loop( 
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_mbload,
)

2022-03-30 01:25:32.387075 Epoch 1, Training loss 2.077747285823383
2022-03-30 01:40:32.219037 Epoch 30, Training loss 0.9063839074748251
2022-03-30 01:55:56.664152 Epoch 60, Training loss 0.7758931545040492
2022-03-30 02:10:53.921817 Epoch 90, Training loss 0.7038726356938062
2022-03-30 02:26:11.919893 Epoch 120, Training loss 0.6541640342348982
2022-03-30 02:41:14.537696 Epoch 150, Training loss 0.616113701401769
2022-03-30 02:56:32.118402 Epoch 180, Training loss 0.5882651089021312
2022-03-30 03:12:03.215099 Epoch 210, Training loss 0.5658418104776641
2022-03-30 03:27:30.533474 Epoch 240, Training loss 0.5450069845446845
2022-03-30 03:42:53.650451 Epoch 270, Training loss 0.5294001597120329
2022-03-30 03:58:15.696856 Epoch 300, Training loss 0.515074334752834


In [ ]:
# Checking the accuracy of model...
train_accuload = torch.utils.data.DataLoader(transformed_cifar10_Train, batch_size = 64, shuffle=False)
Valid_accuload = torch.utils.data.DataLoader(transformed_cifar10_Val, batch_size = 64, shuffle=False)

validate(model, train_accuload , Valid_accuload)

Accuracy train: 0.82
Accuracy val: 0.62
